In [110]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import OrderedDict
import pickle
import pickle5
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score   
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import folium
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import OLSInfluence as influence
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.dummy import DummyRegressor

## block warning
import warnings
warnings.filterwarnings('ignore')

In [111]:
with open("../data/ut_V6_classification-4H.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
3734,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
3735,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
3736,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning
3737,996.5,2018-01-06 17:09:00,2018-01-07 09:45:30,40.000148,-105.282437,2.096847,5,2,326.5,886.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Saturday,Afternoon
3738,52.5,2018-01-07 10:58:00,2018-01-07 11:50:30,40.000148,-105.282437,2.096847,6,2,996.5,326.5,...,0,0,0,0,0,0,0,BOULDER / BASELINE ST1,Sunday,Morning


In [112]:
df.shape

(5110, 102)

# Modeling

In [120]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

In [121]:
## To test model
def test_model(y_test, y_pred):
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    #print('MAE (Mean Absolute Error):', MAE)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    #print('MSE (Mean Squared Error):', MSE)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    #print('RMSE (Root Mean Squared Error):', RMSE)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    
    NRMSE = RMSE/np.mean(y_test)
    return MAE, RMSE, MSE, MAPE, NRMSE

In [124]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
positions = [0,1,2,3,4,5,6]

### Prepare data

In [127]:
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
0,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
1,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
2,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning
3,996.5,2018-01-06 17:09:00,2018-01-07 09:45:30,40.000148,-105.282437,2.096847,5,2,326.5,886.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Saturday,Afternoon
4,52.5,2018-01-07 10:58:00,2018-01-07 11:50:30,40.000148,-105.282437,2.096847,6,2,996.5,326.5,...,0,0,0,0,0,0,0,BOULDER / BASELINE ST1,Sunday,Morning


To make coding easier the names of the models are changed:
- M1: Baseline
- M5: Full

In [129]:
## M1
features1 = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
             'tod_Evening','tod_Midday', 'tod_Morning',
             'Station_Name_BOULDER / N BOULDER REC 1',
             'Station_Name_COMM VITALITY / 1000WALNUT',
             'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + tod
## M5
features5_cat = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
                 'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
                 'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
                 'tod_Evening','tod_Midday', 'tod_Morning',
                 'Station_Name_BOULDER / N BOULDER REC 1',
                 'Station_Name_COMM VITALITY / 1000WALNUT',
                 'Station_Name_COMM VITALITY / 1104 SPRUCE1']
features5_con = ['lag1', 'lag2', 'lag3','near_charge_time_4H', 'near_charge_energy_4H',
                 'charge_time_4H', 'charge_energy_4H','service', 'entertainment', 'food', 
                 'childcare', 'medical', 'education','parking', 'waste-management'] # dow + agg. tod + lag + activ.

In [130]:
split = 0.8

## Quantile regression - Gradiant boosting

#### Get/split data

In [131]:
train, test = np.split(df, [int(split * len(df))])
print("Training shape:",train.shape)
print("Testing shape:", test.shape)

Training shape: (4069, 102)
Testing shape: (1018, 102)


In [132]:
trainS = train[train['y_pred'] == 0]
trainL = train[train['y_pred'] == 1]

testS = test[test['y_pred'] == 0]
testL = test[test['y_pred'] == 1]

In [133]:
print("SHORT: Train: {} Test: {}".format(trainS.shape[0], testS.shape[0]))
print("LONG: Train: {} Test: {}".format(trainL.shape[0], testL.shape[0]))

SHORT: Train: 2430 Test: 628
LONG: Train: 1639 Test: 390


In [ ]:
## Get X 
# Baseline
X1_S_train = trainS[features1]
X1_L_train = trainL[features1]

## Full
# Short
X5_S_train = trainS[features5_cat+features5_con]
scaler = StandardScaler()
X5_S_train[features5_con] = scaler.fit_transform(X5_S_train[features5_con])
X5_S_test = testS[features5_cat+features5_con]
X5_S_test[features5_con] = scaler.transform(X5_S_test[features5_con])
# Full
X5_L_train = trainL[features5_cat+features5_con]
scaler = StandardScaler()
X5_L[features5_con] = scaler.fit_transform(X5_L_train[features5_con])
X5_L_test = testL_train[features5_cat+features5_con]
X5_L_test[features5_con] = scaler.transform(X5_L_test[features5_con])

## Get y - train
y_S_train = trainS['life_time']
y_L_train = trainL['life_time']

## Get y - test
y_S_test = testS['life_time']
y_L_test = testL['life_time']

Data frames for quantile predictions:

In [136]:
quantile_labels = ['5th','25th','50th','75th','95th']
# M1 - Baseline
qr_M1_S = pd.DataFrame(columns=quantile_labels+['true'])
qr_M1_S['true'] = y_S_test
qr_M1_L = pd.DataFrame(columns=quantile_labels+['true'])
qr_M1_L['true'] = y_L_test

# M5 - Full
qr_M5_S = pd.DataFrame(columns=quantile_labels+['true'])
qr_M5_S['true'] = y_S_test
qr_M5_L = pd.DataFrame(columns=quantile_labels+['true'])
qr_M5_L['true'] = y_L_test

In [137]:
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]

In [138]:
results = {}

#### Train M1

In [139]:
## SHORT
for i in range(len(quantiles)):
    print("SHORT, Q: {}".format(quantiles[i]))
    reg = GradientBoostingRegressor(random_state=42, max_depth=3,
                                    n_estimators=300,
                                    min_samples_split=5,
                                    min_samples_leaf=15,
                                    learning_rate=0.01,
                                    loss='quantile', alpha=quantiles[i])
    reg.fit(X1_S_train,y_S_train)
    qr_M1_S[quantile_labels[i]] = reg.predict(X1_S_test)
    
## LONG
for i in range(len(quantiles)):
    print("LONG, Q: {}".format(quantiles[i]))
    reg = GradientBoostingRegressor(random_state=42, max_depth=3,
                                    n_estimators=300,
                                    min_samples_split=5,
                                    min_samples_leaf=15,
                                    learning_rate=0.01,
                                    loss='quantile', alpha=quantiles[i])
    reg.fit(X1_L_train,y_L_train)
    qr_M1_L[quantile_labels[i]] = reg.predict(X1_L_test)
    

SHORT, Q: 0.05
SHORT, Q: 0.25
SHORT, Q: 0.5
SHORT, Q: 0.75
SHORT, Q: 0.95
LONG, Q: 0.05
LONG, Q: 0.25
LONG, Q: 0.5
LONG, Q: 0.75
LONG, Q: 0.95


In [140]:
M1_pred_com = qr_M1_S.append(qr_M1_L)

In [141]:
results['M1'] = {}

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_S['true'],qr_M1_S['50th'])
results['M1']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_S['true'],qr_M1_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_L['true'],qr_M1_L['50th'])
results['M1']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_L['true'],qr_M1_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M1_pred_com['true'],M1_pred_com['50th'])
results['M1']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M1_pred_com['true'],M1_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [142]:
results['M1']

{'S': {'RMSE_test': 417.38513310344973,
  'MAE_test': 238.34760660513396,
  'rsq_test': -0.10119827077707155,
  'MAPE_test': 969.580948798701,
  'NRMSE_test': 1.4858952381406683},
 'L': {'RMSE_test': 435.41306664247867,
  'MAE_test': 303.14467924953624,
  'rsq_test': 0.00030594247622550697,
  'MAPE_test': 1432.0947463619623,
  'NRMSE_test': 0.587185449231718},
 'Com': {'RMSE_test': 424.38221531293595,
  'MAE_test': 263.1716324708676,
  'rsq_test': 0.18281595920702898,
  'MAPE_test': 1146.7718928553531,
  'NRMSE_test': 0.9278823519564449}}

#### Train M5

In [143]:
## SHORT
for i in range(len(quantiles)):
    print("SHORT, Q: {}".format(quantiles[i]))
    reg = GradientBoostingRegressor(random_state=42, max_depth=3,
                                    n_estimators=300,
                                    min_samples_split=5,
                                    min_samples_leaf=15,
                                    learning_rate=0.01,
                                    loss='quantile', alpha=quantiles[i])
    reg.fit(X5_S_train,y_S_train)
    qr_M5_S[quantile_labels[i]] = reg.predict(X5_S_test)
    
## LONG
for i in range(len(quantiles)):
    print("LONG, Q: {}".format(quantiles[i]))
    reg = GradientBoostingRegressor(random_state=42, max_depth=3,
                                    n_estimators=300,
                                    min_samples_split=5,
                                    min_samples_leaf=15,
                                    learning_rate=0.01,
                                    loss='quantile', alpha=quantiles[i])
    reg.fit(X5_L_train,y_L_train)
    qr_M5_L[quantile_labels[i]] = reg.predict(X5_L_test)
    

SHORT, Q: 0.05
SHORT, Q: 0.25
SHORT, Q: 0.5
SHORT, Q: 0.75
SHORT, Q: 0.95
LONG, Q: 0.05
LONG, Q: 0.25
LONG, Q: 0.5
LONG, Q: 0.75
LONG, Q: 0.95


In [144]:
M5_pred_com = qr_M5_S.append(qr_M5_L)

In [145]:
results['M5'] = {}

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_S['true'],qr_M5_S['50th'])
results['M5']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_S['true'],qr_M5_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_L['true'],qr_M5_L['50th'])
results['M5']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_L['true'],qr_M5_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M5_pred_com['true'],M5_pred_com['50th'])
results['M5']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M5_pred_com['true'],M5_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [146]:
results['M5']

{'S': {'RMSE_test': 418.6684070426977,
  'MAE_test': 239.42291970515296,
  'rsq_test': -0.10798007165161194,
  'MAPE_test': 925.7098404158377,
  'NRMSE_test': 1.490463706167741},
 'L': {'RMSE_test': 446.3899664708002,
  'MAE_test': 310.8314390110857,
  'rsq_test': -0.050734623780109844,
  'MAPE_test': 1464.3416518403803,
  'NRMSE_test': 0.6019885783765697},
 'Com': {'RMSE_test': 429.5001348939177,
  'MAE_test': 266.7798180640074,
  'rsq_test': 0.16298713239359586,
  'MAPE_test': 1132.0619096256332,
  'NRMSE_test': 0.9390723290256383}}

### Evaluate QR fit

In [147]:
def ICP(q1, q2, true):
    if (true >= q1) and (true <= q2):
        return 1
    else:
        return 0

In [148]:
######### M1 #########
####### SHORT #######
## 5%-95%
qr_M1_S['MIL5-95'] = qr_M1_S['95th']-qr_M1_S['5th']
qr_M1_S = qr_M1_S.reset_index()
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M1_S.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M1_S['ICP5-95'] = temp

## 25%-75%
qr_M1_S['MIL25-75'] = qr_M1_S['75th']-qr_M1_S['25th']
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M1_S.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M1_S['ICP25-75'] = temp

####### LONG #######
## 5%-95%
qr_M1_L['MIL5-95'] = qr_M1_L['95th']-qr_M1_L['5th']
qr_M1_L = qr_M1_L.reset_index()
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M1_L.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M1_L['ICP5-95'] = temp

## 25%-75%
qr_M1_L['MIL25-75'] = qr_M1_L['75th']-qr_M1_L['25th']
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M1_L.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M1_L['ICP25-75'] = temp

####### COMBINED #######
## 5%-95%
M1_pred_com['MIL5-95'] = M1_pred_com['95th']-M1_pred_com['5th']
M1_pred_com = M1_pred_com.reset_index()
# Get ICP
temp = []
for i in range(len(M1_pred_com)):
    row = M1_pred_com.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
M1_pred_com['ICP5-95'] = temp

## 25%-75%
M1_pred_com['MIL25-75'] = M1_pred_com['75th']-M1_pred_com['25th']
# Get ICP
temp = []
for i in range(len(M1_pred_com)):
    row = M1_pred_com.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
M1_pred_com['ICP25-75'] = temp


In [149]:
######### M5 #########
####### SHORT #######
## 5%-95%
qr_M5_S['MIL5-95'] = qr_M5_S['95th']-qr_M5_S['5th']
qr_M5_S = qr_M5_S.reset_index()
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M5_S.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M5_S['ICP5-95'] = temp

## 25%-75%
qr_M5_S['MIL25-75'] = qr_M5_S['75th']-qr_M5_S['25th']
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M5_S.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M5_S['ICP25-75'] = temp

####### LONG #######
## 5%-95%
qr_M5_L['MIL5-95'] = qr_M5_L['95th']-qr_M5_L['5th']
qr_M5_L = qr_M5_L.reset_index()
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M5_L.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M5_L['ICP5-95'] = temp

## 25%-75%
qr_M5_L['MIL25-75'] = qr_M5_L['75th']-qr_M5_L['25th']
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M5_L.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M5_L['ICP25-75'] = temp

####### COMBINED #######
## 5%-95%
M5_pred_com['MIL5-95'] = M5_pred_com['95th']-M5_pred_com['5th']
M5_pred_com = M5_pred_com.reset_index()
# Get ICP
temp = []
for i in range(len(M5_pred_com)):
    row = M5_pred_com.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
M5_pred_com['ICP5-95'] = temp

## 25%-75%
M5_pred_com['MIL25-75'] = M5_pred_com['75th']-M5_pred_com['25th']
# Get ICP
temp = []
for i in range(len(M5_pred_com)):
    row = M5_pred_com.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
M5_pred_com['ICP25-75'] = temp


In [157]:
qr_M1_S = qr_M1_S.set_index('index')
qr_M1_L = qr_M1_L.set_index('index')
M1_pred_com = M1_pred_com.set_index('index')
qr_M5_S = qr_M5_S.set_index('index')
qr_M5_L = qr_M5_L.set_index('index')
M5_pred_com = M5_pred_com.set_index('index')

In [158]:
qr_M1_S = qr_M1_S.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M1_L = qr_M1_L.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
M1_pred_com = M1_pred_com.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M5_S = qr_M5_S.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M5_L = qr_M5_L.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
M5_pred_com = M5_pred_com.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)

In [162]:
print("Baseline - SHORT")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M1_S[qr_M1_S['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))
    
print("\nBaseline - LONG")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M1_L[qr_M1_L['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

print("\nBaseline - COMBINED")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = M1_pred_com[M1_pred_com['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

Baseline - SHORT
Morning:
 MIL(5-95): 1083.0 ICP(5-95): 0.895   MIL(25-75): 253.0 ICP(25-75): 0.509
Midday:
 MIL(5-95): 1219.0 ICP(5-95): 0.871   MIL(25-75): 225.0 ICP(25-75): 0.476
Afternoon:
 MIL(5-95): 1202.0 ICP(5-95): 0.859   MIL(25-75): 770.0 ICP(25-75): 0.418
Evening:
 MIL(5-95): nan ICP(5-95): nan   MIL(25-75): nan ICP(25-75): nan

Baseline - LONG
Morning:
 MIL(5-95): 1706.0 ICP(5-95): 0.778   MIL(25-75): 796.0 ICP(25-75): 0.444
Midday:
 MIL(5-95): 1658.0 ICP(5-95): 0.837   MIL(25-75): 1016.0 ICP(25-75): 0.245
Afternoon:
 MIL(5-95): 1738.0 ICP(5-95): 0.899   MIL(25-75): 925.0 ICP(25-75): 0.536
Evening:
 MIL(5-95): 1064.0 ICP(5-95): 0.909   MIL(25-75): 318.0 ICP(25-75): 0.555

Baseline - COMBINED
Morning:
 MIL(5-95): 1168.0 ICP(5-95): 0.879   MIL(25-75): 327.0 ICP(25-75): 0.5
Midday:
 MIL(5-95): 1282.0 ICP(5-95): 0.866   MIL(25-75): 338.0 ICP(25-75): 0.443
Afternoon:
 MIL(5-95): 1330.0 ICP(5-95): 0.869   MIL(25-75): 807.0 ICP(25-75): 0.446
Evening:
 MIL(5-95): 1064.0 ICP(5-95): 

In [163]:
print("Full - SHORT")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M5_S[qr_M1_S['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))
    
print("\nFull - LONG")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M5_L[qr_M5_L['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

print("\nFull - COMBINED")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = M5_pred_com[M5_pred_com['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

Full - SHORT
Morning:
 MIL(5-95): 1220.0 ICP(5-95): 0.886   MIL(25-75): 285.0 ICP(25-75): 0.535
Midday:
 MIL(5-95): 1217.0 ICP(5-95): 0.861   MIL(25-75): 237.0 ICP(25-75): 0.476
Afternoon:
 MIL(5-95): 1206.0 ICP(5-95): 0.836   MIL(25-75): 726.0 ICP(25-75): 0.382
Evening:
 MIL(5-95): nan ICP(5-95): nan   MIL(25-75): nan ICP(25-75): nan

Full - LONG
Morning:
 MIL(5-95): 1625.0 ICP(5-95): 0.889   MIL(25-75): 960.0 ICP(25-75): 0.389
Midday:
 MIL(5-95): 1623.0 ICP(5-95): 0.837   MIL(25-75): 968.0 ICP(25-75): 0.245
Afternoon:
 MIL(5-95): 1597.0 ICP(5-95): 0.841   MIL(25-75): 919.0 ICP(25-75): 0.536
Evening:
 MIL(5-95): 1156.0 ICP(5-95): 0.882   MIL(25-75): 341.0 ICP(25-75): 0.614

Full - COMBINED
Morning:
 MIL(5-95): 1275.0 ICP(5-95): 0.886   MIL(25-75): 377.0 ICP(25-75): 0.515
Midday:
 MIL(5-95): 1275.0 ICP(5-95): 0.857   MIL(25-75): 342.0 ICP(25-75): 0.443
Afternoon:
 MIL(5-95): 1300.0 ICP(5-95): 0.837   MIL(25-75): 772.0 ICP(25-75): 0.419
Evening:
 MIL(5-95): 1156.0 ICP(5-95): 0.882   MIL

In [171]:
print("------ SHORT ------ ")
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_S['MIL5-95']),2), 
                                            round(np.sum(qr_M1_S['ICP5-95'])/len(qr_M1_S),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_S['MIL5-95']),2), 
                                            round(np.sum(qr_M5_S['ICP5-95'])/len(qr_M5_S),3)))
print("\n")
print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_S['MIL25-75']),2), 
                                            round(np.sum(qr_M1_S['ICP25-75'])/len(qr_M1_S),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_S['MIL25-75']),2), 
                                            round(np.sum(qr_M5_S['ICP25-75'])/len(qr_M5_S),3)))


print("\n------ LONG ------ ")
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_L['MIL5-95']),2), 
                                            round(np.sum(qr_M1_L['ICP5-95'])/len(qr_M1_L),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_L['MIL5-95']),2), 
                                            round(np.sum(qr_M5_L['ICP5-95'])/len(qr_M5_L),3)))
print("\n")
print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_L['MIL25-75']),2), 
                                            round(np.sum(qr_M1_L['ICP25-75'])/len(qr_M1_L),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_L['MIL25-75']),2), 
                                            round(np.sum(qr_M5_L['ICP25-75'])/len(qr_M5_L),3)))

print("\n------ COMBIEND ------ ")
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(M1_pred_com['MIL5-95']),2), 
                                            round(np.sum(M1_pred_com['ICP5-95'])/len(M1_pred_com),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(M5_pred_com['MIL5-95']),2), 
                                            round(np.sum(M5_pred_com['ICP5-95'])/len(M5_pred_com),3)))
print("\n")
print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(M1_pred_com['MIL25-75']),2), 
                                            round(np.sum(M1_pred_com['ICP25-75'])/len(M1_pred_com),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(M5_pred_com['MIL25-75']),2), 
                                            round(np.sum(M5_pred_com['ICP25-75'])/len(M5_pred_com),3)))

------ SHORT ------ 
Base (5%-95%): MIL: 1188.59 ICP: 0.871
Full (5%-95%): MIL: 1213.95 ICP: 0.857


Base (25%-75%): MIL: 420.83 ICP: 0.462
Full (25%-75%): MIL: 417.17 ICP: 0.454

------ LONG ------ 
Base (5%-95%): MIL: 1287.57 ICP: 0.892
Full (5%-95%): MIL: 1314.6 ICP: 0.869


Base (25%-75%): MIL: 534.73 ICP: 0.508
Full (25%-75%): MIL: 550.81 ICP: 0.544

------ COMBIEND ------ 
Base (5%-95%): MIL: 1226.51 ICP: 0.879
Full (5%-95%): MIL: 1252.51 ICP: 0.861


Base (25%-75%): MIL: 464.47 ICP: 0.479
Full (25%-75%): MIL: 468.37 ICP: 0.488


In [178]:
results = {}
results['M1'] = {}
results['M5'] = {}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_S['true'],qr_M1_S['50th'])
results['M1']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_S['true'],qr_M1_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_L['true'],qr_M1_L['50th'])
results['M1']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_L['true'],qr_M1_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M1_pred_com['true'],M1_pred_com['50th'])
results['M1']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M1_pred_com['true'],M1_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_S['true'],qr_M5_S['50th'])
results['M5']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_S['true'],qr_M5_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_L['true'],qr_M5_L['50th'])
results['M5']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_L['true'],qr_M5_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M5_pred_com['true'],M5_pred_com['50th'])
results['M5']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M5_pred_com['true'],M5_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [187]:
for m in ['M1','M5']:
    #print(m)
    for l in ['S','L','Com']:
        r = results[m][l]
        RMSE = round(r['RMSE_test'],0)
        R2 = round(r['rsq_test'],2)
        MAE = round(r['MAE_test'],0)
        MAPE = round(r['MAPE_test'],0)
        NRMSE = round(r['NRMSE_test'],3)
        
        print("{} - {}:\n R2: {} RMSE: {} MAE: {} MAPE: {} NRMSE: {}".format(m,l,R2,RMSE,MAE,MAPE,NRMSE))  
    print("\n")

M1 - S:
 R2: -0.1 RMSE: 417.0 MAE: 238.0 MAPE: 970.0 NRMSE: 1.486
M1 - L:
 R2: 0.0 RMSE: 435.0 MAE: 303.0 MAPE: 1432.0 NRMSE: 0.587
M1 - Com:
 R2: 0.18 RMSE: 424.0 MAE: 263.0 MAPE: 1147.0 NRMSE: 0.928


M5 - S:
 R2: -0.11 RMSE: 419.0 MAE: 239.0 MAPE: 926.0 NRMSE: 1.49
M5 - L:
 R2: -0.05 RMSE: 446.0 MAE: 311.0 MAPE: 1464.0 NRMSE: 0.602
M5 - Com:
 R2: 0.16 RMSE: 430.0 MAE: 267.0 MAPE: 1132.0 NRMSE: 0.939


